In [3]:
import torch
from mh_nlp.application.use_cases.train_model import TrainModelUseCase

# Détection du hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de : {device}")

# On suppose que 'splits' contient déjà {"train": DTO, "val": DTO, "test": DTO} 
# issus de l'étape précédente (SplitDatasetUseCase)

Utilisation de : cpu


In [4]:
from loguru import logger
from mh_nlp.domain.services.text_cleaner import TextCleaner
from mh_nlp.infrastructure.data.kaggle_repository import KaggleDatasetRepository
from mh_nlp.application.use_cases.build_dataset import BuildCleanDatasetUseCase

# 1.
label_mapping = {
    "Anxiety": 0, 
    "Normal": 1, 
    "Depression": 2,
}

# 2. Initialisation de l'accès aux données et du service de nettoyage
repository = KaggleDatasetRepository(
    csv_path="../data/raw/mental_health.csv",
    label_mapping=label_mapping,
)

cleaner = TextCleaner()

# 3. Injection des dépendances dans le cas d'usage
use_case = BuildCleanDatasetUseCase(
    repository=repository, 
    cleaner=cleaner
)

# 4. Lancement du pipeline et récupération du DatasetDTO
dataset_final = use_case.execute()

2026-01-17 15:14:15.591 | INFO     | mh_nlp.application.use_cases.build_dataset:execute:47 - Démarrage du pipeline BuildCleanDataset.
2026-01-17 15:14:15.591 | INFO     | mh_nlp.infrastructure.data.kaggle_repository:load:33 - Démarrage du chargement des données depuis : ../data/raw/mental_health.csv
2026-01-17 15:14:16.049 | SUCCESS  | mh_nlp.infrastructure.data.kaggle_repository:load:57 - Infrastructure : 1000 paires (Document, Label) créées.
2026-01-17 15:14:16.054 | DEBUG    | mh_nlp.application.use_cases.build_dataset:execute:66 - Traitement unitaire de 1000 documents.
2026-01-17 15:14:16.066 | SUCCESS  | mh_nlp.application.use_cases.build_dataset:execute:92 - Pipeline terminé : 1000 entités synchronisées. (0 supprimées car vides).


In [5]:
from mh_nlp.application.use_cases.split_dataset import SplitDatasetUseCase

# 1. Instanciation du Use Case avec vos ratios
# Ici: 70% Train / 10% Val / 20% Test
splitter = SplitDatasetUseCase(test_size=0.2, val_size=0.1)

# 2. Exécution du split
splits = splitter.execute(dataset_final)

# 3. Récupération des résultats
train_set = splits["train"]
val_set = splits["val"]
test_set = splits["test"]

print(f"Taille Train : {train_set.total_records}")
print(f"Premier document Train : {train_set.documents[0].text}")
print(f"Premier label Train : {train_set.labels[0]}")

2026-01-17 15:14:19.865 | INFO     | mh_nlp.application.use_cases.split_dataset:execute:53 - Découpage du dataset (Total: 1000) | Cible: Test=0.2, Val=0.1
2026-01-17 15:14:19.871 | SUCCESS  | mh_nlp.application.use_cases.split_dataset:execute:86 - Split réussi : Train=700 | Val=100 | Test=200


Taille Train : 700
Premier document Train : anxiety fear overthinking at the same time can be called panic attack right
Premier label Train : 0


In [7]:
from mh_nlp.infrastructure.nlp.hf_tokenizer import HuggingFaceTokenizer
from mh_nlp.infrastructure.models.distilbert_classifier import DistilBertClassifier

# 1. Initialisation
tokenizer_db = HuggingFaceTokenizer("distilbert-base-uncased", max_length=64)
model_db = DistilBertClassifier(
    model_name="distilbert-base-uncased", 
    num_labels=len(label_mapping), 
    tokenizer=tokenizer_db, 
    device=device
)

# 2. Exécution (Le Use Case reste agnostique du modèle)
train_use_case = TrainModelUseCase(model_db)
train_use_case.execute(splits["train"], splits["val"])

2026-01-17 15:17:40.480 | INFO     | mh_nlp.infrastructure.nlp.hf_tokenizer:__init__:34 - Chargement du tokenizer HF : distilbert-base-uncased (max_length=64)
2026-01-17 15:17:41.108 | SUCCESS  | mh_nlp.infrastructure.nlp.hf_tokenizer:__init__:36 - Tokenizer distilbert-base-uncased chargé avec succès.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2026-01-17 15:17:41.370 | DEBUG    | mh_nlp.application.use_cases.train_model:__init__:28 - TrainModelUseCase initialisé avec DistilBertClassifier
2026-01-17 15:17:41.416 | INFO     | mh_nlp.application.use_cases.train_model:execute:50 - Démarrage de l'entraînement : 700 docs (train) | 100 docs (val)
2026-01-17 15:17:41.417 | DEBUG    | mh_nl